In [ ]:
########################################################################
#
# Example of the ptychograpic reconstruction using OpenCL on simulated data
# (c) ESRF 2017-present
# Authors: Vincent Favre-Nicolin <favre@esrf.fr>
#
########################################################################
# floating intensities are yet only supported with OpenCL
import os
os.environ['PYNX_PU'] = 'opencl'

from pylab import *
%matplotlib ipympl
from pynx.ptycho import simulation, shape
import warnings
warnings.simplefilter('ignore')

# Import Ptycho, PtychoData and operators (automatically selecting OpenCL or CUDA)
from pynx.ptycho import *

In [ ]:
##################
# Simulation of the ptychographic data:
n = 256
pixel_size_detector = 55e-6
wavelength = 1.5e-10
detector_distance = 1
obj_info = {'type': 'phase_ampl', 'phase_stretch': pi / 2, 'alpha_win': .2}
probe_info = {'type': 'focus', 'aperture': (30e-6, 30e-6), 'focal_length': .08, 'defocus': 250e-6, 'shape': (n, n)}
probe_info = {'type': 'gauss', 'sigma_pix': (40, 40), 'defocus': 100e-6, 'shape': (n, n)}

# 50 scan positions correspond to 4 turns, 78 to 5 turns, 113 to 6 turns
scan_info = {'type': 'spiral', 'scan_step_pix': 40, 'n_scans': 64}
data_info = {'num_phot_max': 1e9, 'bg': 0, 'wavelength': wavelength, 'detector_distance': detector_distance,
             'detector_pixel_size': pixel_size_detector,
             'noise': 'poisson'}

# Initialisation of the simulation with specified parameters, specific <object>, <probe> or <scan>positions can be passed as:
# s = ptycho.Simulation(obj=<object>, probe=<probe>, scan = <scan>)
# omitting obj_info, probe_info or scan_info (or passing it as empty dictionary "{}")
s = simulation.Simulation(obj_info=obj_info, probe_info=probe_info, scan_info=scan_info, data_info=data_info)

# Data simulation: probe.show(), obj.show(), scan.show() and s.show_illumination_sum() will visualise the integrated total coverage of the beam
s.make_data()

posx, posy = s.scan.values

pixel_size_object = wavelength * detector_distance / pixel_size_detector / n

ampl = s.amplitude.values  # square root of the measured diffraction pattern intensity

##################
# Size of the reconstructed object (obj)
nyo, nxo = shape.calc_obj_shape(posx, posy, ampl.shape[1:])

# Initial object
# obj_init_info = {'type':'flat','shape':(nx,ny)}
obj_init_info = {'type': 'random', 'range': (0, 1, 0, 0.5), 'shape': (nyo, nxo)}
# Initial probe
probe_init_info = {'type': 'focus', 'aperture': (20e-6, 20e-6), 'focal_length': .08, 'defocus': 50e-6, 'shape': (n, n)}
data_info = {'wavelength': wavelength, 'detector_distance': detector_distance,
             'detector_pixel_size': pixel_size_detector}
init = simulation.Simulation(obj_info=obj_init_info, probe_info=probe_init_info, data_info=data_info)

init.make_obj()
init.make_probe()

In [ ]:
# Add top-up type scaling of the intensity (we should also use Poisson statistics for good measure...)
iobs = ampl ** 2
iobs_f = []
for i in range(len(ampl)):
    iobs_f.append(np.exp(-(i%10)/40))
    iobs[i] *= iobs_f[-1]
iobs_f = np.array(iobs_f)
iobs_f /= iobs_f.mean()

data = PtychoData(iobs=iobs, positions=(posx * pixel_size_object, posy * pixel_size_object), 
                  detector_distance=1, mask=None, pixel_size_detector=55e-6, wavelength=1.5e-10)

if True:
    p = Ptycho(probe=s.probe.values, obj=init.obj.values, data=data, background=None) # Random object start
else:
    p = Ptycho(probe=s.probe.values, obj=s.obj.values, data=data, background=None)  #
# Initial scaling of object and probe
p = ScaleObjProbe() * p

In [ ]:
plt.figure(figsize=(9.5,6))
p = DM(update_object=True, update_probe=True, calc_llk=10)**40 * p
#p = AP(update_object=True, update_probe=False, calc_llk=10)**40 * p
p = ML(update_object=True, update_probe=False, calc_llk=10)**20 * p
p = ShowObjProbe() * p

In [ ]:
p = ShowObjProbe() * AP(update_object=True, update_probe=False, floating_intensity=True, calc_llk=10)**40 * p

In [ ]:
p = ML(update_object=True, update_probe=False, floating_intensity=True, calc_llk=10) ** 50 * p

In [ ]:
p.from_pu()
plt.figure()
plt.plot(p.data.scale,'k-', label='scale_calc')
iobs_f /= iobs_f.mean()
plt.plot(iobs_f, 'r-', label='scale_simul')